<a href="https://colab.research.google.com/github/weso500/QMLPublicationRuns/blob/main/QuantumClose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install qiskit-machine-learning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 7.2 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.3
    Uninstalling scipy-1.16.3:
      Successfully uninstalled scipy-1.16.3


In [2]:
pip install pyreadr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.3/418.3 kB 6.8 MB/s eta 0:00:00


In [3]:
import pyreadr
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [4]:
import pandas as pd
import numpy as np

# Load the training data
train_df = pd.read_csv('/content/drive/MyDrive/QMLIIOT/Train_11_final.csv')

# Load the test data
test_df = pd.read_csv('/content/drive/MyDrive/QMLIIOT/Test_11_final.csv')


In [5]:
pip install qiskit-ibm-runtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.4/377.4 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 13.2 MB/s eta 0:00:00


In [6]:
from qiskit_ibm_runtime import QiskitRuntimeService

QiskitRuntimeService.save_account(
token="1YoD2JW0lRfK4dJNmWZf2fQTV7zpzxTWe4UKboQmwLVd", # Use the 44-character API_KEY you created and saved from the IBM Quantum Platform Home dashboard
instance="open-instance",
overwrite=True
)

In [7]:
from qiskit_ibm_runtime import QiskitRuntimeService

# Run every time you need the service
service = QiskitRuntimeService()

In [8]:
pip install qiskit_algorithms

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.8/327.8 kB 8.5 MB/s eta 0:00:00


In [9]:
!pip install -U qiskit-algorithms

In [11]:
import qiskit_algorithms
print("Qiskit Algorithms version:", qiskit_algorithms.__version__)


Qiskit Algorithms version: 0.4.0


In [40]:
# ============================================================
# Quantum Nyström OCSVM using IBM Runtime Backend
# - Uses FidelityQuantumKernel evaluated on IBM real backend
# - Falls back to local statevector simulator if needed
# ============================================================

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from qiskit.primitives import Sampler, BackendSamplerV2
from sklearn.svm import OneClassSVM
from qiskit_ibm_runtime import SamplerV2 as Sampler
from sklearn import metrics
from sklearn.kernel_approximation import Nystroem
from joblib import Parallel, delayed


from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.primitives import BackendSampler
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit.circuit.library import ZFeatureMap
from qiskit_algorithms.state_fidelities import ComputeUncompute
from qiskit import transpile

# ============================================================
# --- IBM Quantum Runtime Setup ---
# ============================================================
service = QiskitRuntimeService(channel="ibm_quantum_platform", instance="open-instance")

# Pick a real backend (you can change this to 'ibm_osaka', 'ibm_brisbane', etc.)
backend = service.backend("ibm_brisbane")
print(backend.operation_names)

sampler = Sampler(backend, options={"default_shots": 4096})

fidelity = ComputeUncompute(sampler=sampler)

raw_map = ZFeatureMap(feature_dimension=train_df.shape[1], reps=1, parameter_prefix="θ" )
# Transpile for your backend topology
feature_map = transpile(raw_map,
                        backend=backend,
                        basis_gates=backend.operation_names,
                        optimization_level=1)
print(feature_map.count_ops())
kernel = FidelityQuantumKernel(feature_map=feature_map, fidelity=fidelity)

# ============================================================
# --- Statistical Helpers ---
# ============================================================
def mean_ci(arr, alpha=0.05):
    arr = np.asarray(arr, dtype=float)
    n = len(arr)
    m = np.mean(arr)
    s = np.std(arr, ddof=1) if n > 1 else 0.0
    z = 1.96
    half = z * (s / np.sqrt(max(n, 1)))
    return m, m - half, m + half


def youden_threshold(y_true, scores):
    fpr, tpr, thr = metrics.roc_curve(y_true, scores, pos_label=1)
    mask = np.isfinite(thr)
    fpr, tpr, thr = fpr[mask], tpr[mask], thr[mask]
    j = tpr - fpr
    if len(j) == 0:
        return np.median(scores)
    idx = np.argmax(j)
    return thr[idx]

# ============================================================
# --- One Experimental Trial ---
# ============================================================
def run_trial(i, fault_id=1, n_train_normals=100, n_test_normals=70, n_test_faulty=5,
              nystrom_components=32, nu=0.05):
    rng = np.random.RandomState(i)
    sc = StandardScaler()

    # Randomly sample subsets
    train_features = train_df.sample(n=n_train_normals, random_state=i).to_numpy()
    test_features_normal = test_df.iloc[:500].sample(n=n_test_normals, random_state=i).to_numpy()
    test_features_anomaly = test_df.iloc[500:600].sample(n=n_test_faulty, random_state=i).to_numpy()
    test_features = np.concatenate((test_features_normal, test_features_anomaly))
    y_true = np.concatenate((np.zeros(n_test_normals), np.ones(n_test_faulty)))

    # --- Nyström approximation of quantum kernel ---
    n_components = min(nystrom_components, len(train_features) - 1)
    Phi = Nystroem(
        kernel=lambda A, B=None: kernel.evaluate(x_vec=A, y_vec=B),
        n_components=n_components,
        random_state=i
    )

    # Compute feature mappings
    Ztr = Phi.fit_transform(train_features)
    Zte = Phi.transform(test_features)

    # Gram matrices
    Ktr = Ztr @ Ztr.T
    Kte = Zte @ Ztr.T

    # Train OCSVM
    ocsvm = OneClassSVM(kernel="precomputed", nu=nu)
    ocsvm.fit(Ktr)
    scores = -ocsvm.decision_function(Kte)

    # Metrics
    roc_auc = metrics.roc_auc_score(y_true, scores)
    pr_auc = metrics.average_precision_score(y_true, scores)
    thr = youden_threshold(y_true, scores)
    y_pred = (scores >= thr).astype(int)

    precision = metrics.precision_score(y_true, y_pred, zero_division=0)
    recall = metrics.recall_score(y_true, y_pred, zero_division=0)
    f1 = metrics.f1_score(y_true, y_pred, zero_division=0)
    accuracy = metrics.accuracy_score(y_true, y_pred)

    return dict(roc_auc=roc_auc, pr_auc=pr_auc, precision=precision,
                recall=recall, f1=f1, accuracy=accuracy, threshold=thr)

# ============================================================
# --- Run Experiments ---
# ============================================================
N_RUNS = 1   # reduce for hardware; each call runs quantum circuits!
results = [run_trial(i) for i in range(N_RUNS)]
df = pd.DataFrame(results)

# ============================================================
# --- Summaries ---
# ============================================================
summary_rows = []
for metric in ["roc_auc", "pr_auc", "precision", "recall", "f1", "accuracy", "threshold"]:
    m, lo, hi = mean_ci(df[metric].values)
    summary_rows.append({"metric": metric, "mean": m, "ci95_lo": lo, "ci95_hi": hi})

summary = pd.DataFrame(summary_rows).set_index("metric")
print("\n=== Cross-run metrics (mean ± 95% CI) ===")
for metric, row in summary.iterrows():
    if metric == "threshold":
        print(f"{metric:>9}: {row['mean']:.4f}  (95% CI: {row['ci95_lo']:.4f} .. {row['ci95_hi']:.4f})")
    else:
        print(f"{metric:>9}: {row['mean']:.3f}  (95% CI: {row['ci95_lo']:.3f} .. {row['ci95_hi']:.3f})")

['if_else', 'x', 'delay', 'sx', 'measure', 'id', 'reset', 'rz', 'ecr']
OrderedDict({'rz': 33, 'sx': 11})


IBMInputValueError: 'The instruction sxdg on qubits (10,) is not supported by the target system. Circuits that do not match the target hardware definition are no longer supported after March 4, 2024. See the transpilation documentation (https://quantum.cloud.ibm.com/docs/guides/transpile) for instructions to transform circuits and the primitive examples (https://quantum.cloud.ibm.com/docs/guides/primitives-examples) to see this coupled with operator transformations.'

In [37]:
# ============================================================
# Quantum Nyström OCSVM on IBM Hardware (Runtime V2, ISA-safe)
# - Native feature map with only rz/sx/cz (no sxdg ever)
# - Strict transpilation to backend.operation_names
# ============================================================

import numpy as np
import pandas as pd
from copy import deepcopy
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM
from sklearn import metrics
from sklearn.kernel_approximation import Nystroem

from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2
from qiskit_algorithms.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit import QuantumCircuit, transpile

# -----------------------------
# IBM Runtime setup
# -----------------------------
service = QiskitRuntimeService(channel="ibm_quantum_platform")
instances = service.instances()
instance = instances[0] if instances else None
service = QiskitRuntimeService(channel="ibm_quantum_platform", instance="open-instance")

# Choose a backend that supports rz/sx/cz
backend_name = None
for b in service.backends():
    try:
        ops = set(b.operation_names)
        if {"rz", "sx", "cz"}.issubset(ops):
            backend_name = b.name
            break
    except Exception:
        pass
if backend_name is None:
    raise RuntimeError("No suitable backend with rz/sx/cz support found.")
backend = service.backend(backend_name)
print("Using backend:", backend.name, "ops:", backend.operation_names)

sampler = SamplerV2(backend, options={"default_shots": 4096})
fidelity = ComputeUncompute(sampler=sampler)

# -----------------------------
# Native feature map (rz/sx/cz only)
# -----------------------------
def build_native_feature_map(n_features: int, reps: int = 1, parameter_prefix: str = "θ"):
    """
    Angle-encoding on each qubit with (rz(θ_i) -> sx) followed by linear CZ entanglement.
    Only uses rz, sx, cz so it's ISA-safe on backends listing those ops.
    """
    qc = QuantumCircuit(n_features, name="NativeFM")
    # create parameters θ_0..θ_{n-1}
    from qiskit.circuit import ParameterVector
    thetas = ParameterVector(parameter_prefix, n_features)

    for _ in range(reps):
        # single-qubit feature layer
        for q in range(n_features):
            qc.rz(thetas[q], q)
            qc.sx(q)
        # linear entanglement with CZ
        for q in range(n_features - 1):
            qc.cz(q, q + 1)
    return qc, thetas

RAW_FEATURE_DIM = train_df.shape[1]
raw_map, theta_vec = build_native_feature_map(RAW_FEATURE_DIM, reps=1, parameter_prefix="θ")

def transpile_for_backend(circ, backend):
    return transpile(
        deepcopy(circ),
        backend=backend,
        basis_gates=backend.operation_names,    # enforce ISA
        coupling_map=backend.coupling_map,
        optimization_level=3
    )

# transpile a template once; deepcopy per kernel to avoid param clashes
transpiled_map_template = transpile_for_backend(raw_map, backend)
print("Transpiled ops:", transpiled_map_template.count_ops())  # should NOT include 'sxdg'

def make_kernel():
    # Deepcopy to ensure fresh parameters for each evaluate()
    fm = deepcopy(transpiled_map_template)
    return FidelityQuantumKernel(feature_map=fm, fidelity=fidelity)

# -----------------------------
# Helpers
# -----------------------------
def mean_ci(arr, alpha=0.05):
    arr = np.asarray(arr, dtype=float)
    n = len(arr)
    m = np.mean(arr)
    s = np.std(arr, ddof=1) if n > 1 else 0.0
    z = 1.96
    half = z * (s / np.sqrt(max(n, 1)))
    return m, m - half, m + half

def youden_threshold(y_true, scores):
    fpr, tpr, thr = metrics.roc_curve(y_true, scores, pos_label=1)
    mask = np.isfinite(thr)
    fpr, tpr, thr = fpr[mask], tpr[mask], thr[mask]
    j = tpr - fpr
    if len(j) == 0:
        return np.median(scores)
    idx = np.argmax(j)
    return thr[idx]

# -----------------------------
# One trial
# -----------------------------
def run_trial(i, fault_id=1, n_train_normals=100, n_test_normals=70, n_test_faulty=5,
              nystrom_components=32, nu=0.05):
    rng = np.random.RandomState(i)
    sc = StandardScaler()

    # Sample data
    train_features = train_df.sample(n=n_train_normals, random_state=i).to_numpy()
    test_features_normal = test_df.iloc[:500].sample(n=n_test_normals, random_state=i).to_numpy()
    test_features_anomaly = test_df.iloc[500:600].sample(n=n_test_faulty, random_state=i).to_numpy()
    test_features = np.concatenate((test_features_normal, test_features_anomaly))
    y_true = np.concatenate((np.zeros(n_test_normals), np.ones(n_test_faulty)))

    # Build kernel (fresh FM copy)
    kernel = make_kernel()

    # Nyström approx
    n_components = min(nystrom_components, len(train_features) - 1)
    Phi = Nystroem(
        kernel=lambda A, B=None: kernel.evaluate(x_vec=A, y_vec=B),
        n_components=n_components,
        random_state=i
    )

    Ztr = Phi.fit_transform(train_features)
    Zte = Phi.transform(test_features)

    # Gram matrices
    Ktr = Ztr @ Ztr.T
    Kte = Zte @ Ztr.T

    # OCSVM
    ocsvm = OneClassSVM(kernel="precomputed", nu=nu)
    ocsvm.fit(Ktr)
    scores = -ocsvm.decision_function(Kte)

    # Metrics
    roc_auc = metrics.roc_auc_score(y_true, scores)
    pr_auc  = metrics.average_precision_score(y_true, scores)
    thr     = youden_threshold(y_true, scores)
    y_pred  = (scores >= thr).astype(int)

    precision = metrics.precision_score(y_true, y_pred, zero_division=0)
    recall    = metrics.recall_score(y_true, y_pred, zero_division=0)
    f1        = metrics.f1_score(y_true, y_pred, zero_division=0)
    accuracy  = metrics.accuracy_score(y_true, y_pred)

    return dict(roc_auc=roc_auc, pr_auc=pr_auc, precision=precision,
                recall=recall, f1=f1, accuracy=accuracy, threshold=thr)

# -----------------------------
# Run + summarize
# -----------------------------
N_RUNS = 1   # keep small on hardware
results = [run_trial(i) for i in range(N_RUNS)]
df = pd.DataFrame(results)

summary_rows = []
for metric in ["roc_auc", "pr_auc", "precision", "recall", "f1", "accuracy", "threshold"]:
    m, lo, hi = mean_ci(df[metric].values)
    summary_rows.append({"metric": metric, "mean": m, "ci95_lo": lo, "ci95_hi": hi})
summary = pd.DataFrame(summary_rows).set_index("metric")

print("\n=== Cross-run metrics (mean ± 95% CI) ===")
for metric, row in summary.iterrows():
    if metric == "threshold":
        print(f"{metric:>9}: {row['mean']:.4f}  (95% CI: {row['ci95_lo']:.4f} .. {row['ci95_hi']:.4f})")
    else:
        print(f"{metric:>9}: {row['mean']:.3f}  (95% CI: {row['ci95_lo']:.3f} .. {row['ci95_hi']:.3f})")


Using backend: ibm_fez ops: ['if_else', 'x', 'delay', 'cz', 'sx', 'measure', 'id', 'reset', 'rz']
Transpiled ops: OrderedDict({'rz': 11, 'sx': 11, 'cz': 10})


IBMInputValueError: 'The instruction sxdg on qubits (129,) is not supported by the target system. Circuits that do not match the target hardware definition are no longer supported after March 4, 2024. See the transpilation documentation (https://quantum.cloud.ibm.com/docs/guides/transpile) for instructions to transform circuits and the primitive examples (https://quantum.cloud.ibm.com/docs/guides/primitives-examples) to see this coupled with operator transformations.'

In [38]:
for b in service.backends():
    if "sxdg" in b.operation_names:
        print(b.name)